In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Data loading and preprocessing
data_dir = '/kaggle/input/pothole-detection-dataset'
normal_images = []
for img_file in os.listdir(os.path.join(data_dir, 'normal')):
    img_path = os.path.join(data_dir, 'normal', img_file)
    if os.path.isfile(img_path):
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            normal_images.append(img)

pothole_images = []
for img_file in os.listdir(os.path.join(data_dir, 'potholes')):
    img_path = os.path.join(data_dir, 'potholes', img_file)
    if os.path.isfile(img_path):
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            pothole_images.append(img)

# Normalizing pixel values
normal_images = [img / 255.0 for img in normal_images]
pothole_images = [img / 255.0 for img in pothole_images]

# Assigning labels
X = np.array(normal_images + pothole_images)
y = np.array([0]*len(normal_images) + [1]*len(pothole_images))

In [7]:

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model building
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
17/17 [==============================] - 5s 208ms/step - loss: 0.8690 - accuracy: 0.5331 - val_loss: 0.6622 - val_accuracy: 0.6985
Epoch 2/10
17/17 [==============================] - 3s 178ms/step - loss: 0.6122 - accuracy: 0.7592 - val_loss: 0.5136 - val_accuracy: 0.7868
Epoch 3/10
17/17 [==============================] - 3s 177ms/step - loss: 0.4006 - accuracy: 0.8585 - val_loss: 0.3484 - val_accuracy: 0.8529
Epoch 4/10
17/17 [==============================] - 3s 175ms/step - loss: 0.2503 - accuracy: 0.9136 - val_loss: 0.2433 - val_accuracy: 0.9118
Epoch 5/10
17/17 [==============================] - 3s 176ms/step - loss: 0.2063 - accuracy: 0.9228 - val_loss: 0.2571 - val_accuracy: 0.8971
Epoch 6/10
17/17 [==============================] - 3s 177ms/step - loss: 0.1640 - accuracy: 0.9393 - val_loss: 0.1964 - val_accuracy: 0.9265
Epoch 7/10
17/17 [==============================] - 3s 191ms/step - loss: 0.1362 - accuracy: 0.9504 - val_loss: 0.2930 - val_accuracy: 0.8750
Epoch 

In [22]:
import cv2
import numpy as np

# Loading the image
img_path = '/kaggle/input/sample3/sample2.jpg'
img = cv2.imread(img_path)

if img is not None:
    # Resizeing and normalize the image
    img = cv2.resize(img, (128, 128))
    img = img / 255.0

    # Reshaping the image to match the model's input shape
    img = np.expand_dims(img, axis=0)

    # Make a prediction
    prediction = model.predict(img)

    # Print the prediction
    print(prediction)
else:
    print(f"Failed to load image from path: {img_path}")


1/1 [==============================] - 0s 22ms/step
[[0.42607275]]


In [23]:
# Save the model
model.save('simple_pothole.keras')